library imports

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

grab text

In [ ]:
file = tf.keras.utils.get_file('raw.txt', 'https://www.gutenberg.org/files/2554/2554-0.txt')

explore text

In [ ]:
# Read, then decode for py2 compat.
text = open(file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

first characters

In [ ]:
text = text[5540:]
print(text[:250])

get a list of unique characters

In [ ]:
# The unique characters in the file
unique = sorted(set(text))
print(f'{len(unique)} unique characters')

vectorize text (convert strings to a numerical representation)

In [ ]:
# split text into tokens
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

add stringlookup layer

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(unique), mask_token=None)
    
ids = ids_from_chars(chars)
ids

initialize layer for reverse operation (vector to text)

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
    
chars = chars_from_ids(ids)
chars

join bit chars into strings with the following function:

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

define the inverse operation:

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

vectorize all text

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

dump text into a dataset

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

convert individual characters to sequences of size 100

In [63]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\n' b'\r' b'\n' b'\r' b'\n' b'C' b'R' b'I' b'M' b'E' b' ' b'A' b'N'
 b'D' b' ' b'P' b'U' b'N' b'I' b'S' b'H' b'M' b'E' b'N' b'T' b'\r' b'\n'
 b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'P' b'A' b'R' b'T' b' '
 b'I' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'C' b'H' b'A' b'P'
 b'T' b'E' b'R' b' ' b'I' b'\r' b'\n' b'\r' b'\n' b'O' b'n' b' ' b'a' b'n'
 b' ' b'e' b'x' b'c' b'e' b'p' b't' b'i' b'o' b'n' b'a' b'l' b'l' b'y'
 b' ' b'h' b'o' b't' b' ' b'e' b'v' b'e' b'n' b'i' b'n' b'g' b' ' b'e'
 b'a' b'r' b'l' b'y' b' ' b'i'], shape=(101,), dtype=string)


join sequences back into strings

In [64]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\n\r\n\r\nCRIME AND PUNISHMENT\r\n\r\n\r\n\r\n\r\nPART I\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\nOn an exceptionally hot evening early i'
b'n July a young man came out of\r\nthe garret in which he lodged in S. Place and walked slowly, as thoug'
b'h\r\nin hesitation, towards K. bridge.\r\n\r\nHe had successfully avoided meeting his landlady on the stair'
b'case. His\r\ngarret was under the roof of a high, five-storied house and was more\r\nlike a cupboard than'
b' a room. The landlady who provided him with garret,\r\ndinners, and attendance, lived on the floor belo'


define 'split_input_target', a function which, at each time step, takes input as the current character and label as the next character

In [65]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

map function across our sequences, save to dataset

In [67]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\n\r\n\r\nCRIME AND PUNISHMENT\r\n\r\n\r\n\r\n\r\nPART I\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\nOn an exceptionally hot evening early '
Target: b'\r\n\r\nCRIME AND PUNISHMENT\r\n\r\n\r\n\r\n\r\nPART I\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\nOn an exceptionally hot evening early i'


shuffle and pack data into batches for training

In [68]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

build 3-layer model

In [69]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [70]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [71]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [72]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 100) # (batch_size, sequence_length, vocab_size)


In [73]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  25600     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  102500    
                                                                 
Total params: 4,066,404
Trainable params: 4,066,404
Non-trainable params: 0
_________________________________________________________________


make predictions by sampling from output distribution

In [75]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

sampled_indices

array([ 8, 15, 37, 24, 40,  5, 48, 54, 51,  4, 39, 49, 99, 91, 92, 24, 74,
       13, 11, 80,  5, 73, 62, 24, 38,  7,  3, 49, 16, 11, 37, 78,  4, 95,
       55, 78, 84, 38, 94, 61, 35, 29, 20, 49, 55, 95, 76, 85, 13, 15, 67,
       32,  4, 38, 10, 39, 54, 54, 56, 94,  9, 94, 61, 58, 85, 37, 38, 40,
       82, 74,  0, 64, 69, 51, 59, 43, 97, 36,  5, 87,  3, 74, 15, 28, 13,
       90, 20,  7, 10, 69, 38, 70, 93, 95, 12, 97, 25, 91, 93,  9])

decode

In [76]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' all\r\npretty, in fact I am told positively ugly... and such an invalid... and\r\nqueer. But she seems '

Next Char Predictions:
 b'\'/I8L"TZW!KU\xe2\x80\x9d\xc3\xae\xc3\xaf8q-*w"pe8J% U0*Iu!\xc3\xbc[u\xc3\xa0J\xc3\xb6dGA4U[\xc3\xbcs\xc3\xa4-/jD!J)KZZ]\xc3\xb6(\xc3\xb6da\xc3\xa4IJLyq[UNK]glWbO\xe2\x80\x99H"\xc3\xa7 q/?-\xc3\xaa4%)lJm\xc3\xb4\xc3\xbc,\xe2\x80\x999\xc3\xae\xc3\xb4('


add optimizer, loss function

In [77]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [78]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 100)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.606567, shape=(), dtype=float32)


In [79]:
#should be roughly equal to vocab size, 99
tf.exp(example_batch_mean_loss).numpy()

100.13977

compile model

In [80]:
model.compile(optimizer='adam', loss=loss)

add checkpoints

In [81]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

train for 20 epochs

In [82]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
 16/181 [=>............................] - ETA: 43:44 - loss: 4.3313

KeyboardInterrupt: 

generate text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)